In [1]:
import json
import requests
import numpy as np
import pandas as pd

from transformers import BertTokenizer

df = pd.read_csv('datasets/cnn_content_2023-11-11.csv')
df = df[['berita']]

c:\Users\LENOVO\GitHub\bangkit_capstone\VENU-ML\HOAX_DETECTION\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
df_test = df.sample()

# df_test.cleaned.values
len(df_test['berita'].values[0].split())

272

In [23]:
df_test['berita'].values[0]

'Undang-undang Nomor 7 Tahun 2017 tentang Pemilu melarang tim kampanye pasangan capres-cawapres mengikutsertakan hakim Mahkamah Konstitusi (MK) hingga komisaris serta direksi Badan Usaha Milik Negara (BUMN) atau BUMD dalam kegiatan kampanye Pilpres 2024. Hal ini diatur dalam UU Pemilu Pasal 280 ayat (2) huruf a dan d yang berbunyi sebagai berikut. "Pelaksana dan/atau tim kampanye dalam kegiatan Kampanye Pemilu dilarang mengikutsertakan: a. Ketua, wakil ketua, ketua muda, hakim agung pada Mahkamah Agung, dan hakim pada semua badan peradilan di bawah Mahkamah Agung, dan hakim konstitusi pada Mahkamah Konstitusi,"   "d. direksi, komisaris, dewan pengawas dan karyawan badan usaha milik negara/badan usaha milik daerah."  Hakim MK dan komisaris BUMN/BUMD juga dilarang menjadi pelaksana maupun tim sukses kampanye paslon tertentu. Aturan ini pun berlaku bagi ketua, wakil ketua, dan anggota BPK; gubernur, deputi gubernur senior, dan deputi gubernur BI. Kemudian pejabat negara yang menjabat seba

In [24]:
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-lite-base-p1')

max_len = 70

tokenized = tokenizer(
    text=df_test['berita'].tolist(),
    add_special_tokens=True,
    max_length=max_len,
    truncation=True,
    padding='max_length',
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
)

tokenized

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


{'input_ids': <tf.Tensor: shape=(1, 70), dtype=int32, numpy=
array([[    2,  9363, 29947,  9363,  1288,   680,   262,  3951,   416,
         4482,  8051,   511,  6343,  2038, 15994, 29947, 25951, 11245,
        22672,  4887, 10354,  7343, 29942,  8221, 29943,   733, 13741,
          501, 12403,  1269,  1062,  2318,   664, 29942,  4098, 29943,
          158, 23514,   112,   845,  6343, 15140, 23917, 29860, 29948,
          269,    92,  4573,   112,  3719,  4482,  2051, 24798,  1862,
        29942,   109, 29943,  3307,   253,    41,     6,    34, 10388,
          242,   713, 29948, 29936,  7416,    41,     3]])>, 'attention_mask': <tf.Tensor: shape=(1, 70), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1]])>}

In [25]:
X1 = np.squeeze(tokenized['attention_mask']._numpy())
X2 = np.squeeze(tokenized['input_ids']._numpy())


In [26]:
data1 = json.dumps({"signature_name": "serving_default",
                   "instances": [{'attention_mask':X1.tolist(),'input_ids':X2.tolist()}]})
# both x and y are numpy 2d array 

json_response = requests.post('http://localhost:8501/v1/models/NLP:predict',
                               data=data1)

pred = json.loads(json_response.text)

print(pred)

{'predictions': [[0.0207458735]]}


In [20]:
!saved_model_cli show --dir C:/Users/LENOVO/GitHub/bangkit_capstone/VENU-ML/HOAX_DETECTION/test_deploy/model/1 --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['attention_mask'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 70)
        name: serving_default_attention_mask:0
    inputs['input_ids'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 70)
        name: serving_default_input_ids:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_2'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predic

2023-12-10 10:04:53.302482: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE SSE2 SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "c:\users\lenovo\github\bangkit_capstone\venu-ml\hoax_detection\venv\lib\site-packages\tensorflow\python\training\py_checkpoint_reader.py", line 92, in NewCheckpointReader
    return CheckpointReader(compat.as_bytes(filepattern))
RuntimeError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for C:/Users/LENOVO/GitHub/bangkit_capstone/VENU-ML/HOAX_DETECTION/test_deploy/model/2\variables\variables

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\lenovo\github\bangkit_capstone\venu-ml\hoax_detection\venv\lib\


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['attention_mask'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 70)
        name: serving_default_attention_mask:0
    inputs['input_ids'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 70)
        name: serving_default_input_ids:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_2'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predic